In [50]:
import os
import pandas as pd
import numpy as np
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split

# Define the path to the "data" folder (one level up from "notebooks")
data_folder = os.path.join(os.path.dirname(os.getcwd()), "data")

# Define input and output file paths
input_file = os.path.join(data_folder, "ecom_transactions.csv")
output_file = os.path.join(data_folder, "cleaned_ecom_transactions.csv")

# Load dataset
df = pd.read_csv(input_file)

# Display the first few rows before cleaning
print("Before Cleaning:")
print(df.head())

### 1. HANDLE MISSING VALUES ###
# Convert 'Price' to numeric, setting errors to NaN
df["Price"] = pd.to_numeric(df["Price"], errors="coerce")

# Fill missing Product Categories with the most common one
df.loc[:, "Product Category"] = df["Product Category"].fillna(df["Product Category"].mode()[0])

# Convert 'Transaction Date' to datetime, forcing errors to NaT
df["Transaction Date"] = pd.to_datetime(df["Transaction Date"], errors="coerce")

# Convert 'Quantity' to integer (fill NaN with 1, assuming a single order)
df.loc[:, "Quantity"] = pd.to_numeric(df["Quantity"], errors="coerce").fillna(1).astype(int)

# Use linear regression to fill missing Price values
valid_rows = df.dropna(subset=["Price"])  # Rows with known Price
missing_rows = df[df["Price"].isna()]  # Rows with missing Price

if not missing_rows.empty:
    # Prepare features for regression
    X = valid_rows[["Quantity"]]
    y = valid_rows["Price"]
    
    model = LinearRegression()
    model.fit(X, y)
    
    # Predict missing Prices
    df.loc[df["Price"].isna(), "Price"] = model.predict(missing_rows[["Quantity"]])

### 2. REMOVE DUPLICATES ###
df = df.drop_duplicates(subset=["Transaction ID"], keep="first")

### 3. HANDLE OUTLIERS ###
# Remove extreme outliers in Price (above 99th percentile)
upper_limit = df["Price"].quantile(0.99)
df = df[df["Price"] <= upper_limit]

# Remove extreme Quantity outliers (above 1000)
df = df[df["Quantity"] <= 1000]

### 4. FIX CATEGORICAL VALUES ###
# Standardize Product Category names (capitalize each word)
df.loc[:, "Product Category"] = df["Product Category"].str.title()

### 5. REMOVE INCORRECT VALUES ###
# Drop rows where Price or Quantity is negative
df = df[(df["Price"] > 0) & (df["Quantity"] > 0)]

# Fill missing Customer Ratings with the median value
df.loc[:, "Customer Rating"] = df["Customer Rating"].fillna(df["Customer Rating"].median())

# Fill missing Payment Methods with the most common one
df.loc[:, "Payment Method"] = df["Payment Method"].fillna(df["Payment Method"].mode()[0])

### 6. CHECK FOR BLANK VALUES ###
def check_blank_cells(df):
    """Checks for blank cells in the dataset and prints the results."""
    blank_counts = (df == "").sum() + (df == " ").sum()
    blank_columns = blank_counts[blank_counts > 0]
    
    if blank_columns.empty:
        print("\n✅ No blank cells found in the dataset.")
    else:
        from IPython.display import display  # Import for Jupyter display
        print("\n⚠️ Blank cells detected in the following columns:")
        display(blank_columns.to_frame(name="Blank Count"))  # Better display format

# Run the function on the cleaned DataFrame
check_blank_cells(df)

# Display cleaned dataset
print("\nAfter Cleaning:")
print(df.head())

# Save the cleaned dataset
df.to_csv(output_file, index=False)
print(f"\n✅ Data Cleaning Completed! Cleaned file saved at '{output_file}'")

Before Cleaning:
                         Transaction ID                           Customer ID  \
0  539db039-37c0-4f91-b6fa-079af50ee7fd  c3b9ac53-4a42-46ac-9850-f46605e4e94a   
1  2c37f0fc-0ca2-4f93-97bf-659fa96788f1  1b2acf0b-cb58-44c4-a272-6f01d5b0116e   
2  037d7859-03e2-45a0-ad46-80d8ab92fbd4  4a636dcd-686b-45fc-ba6e-54056dc273fb   
3  84d6a5ff-a05c-4d7c-9fb5-1106a1594a97  b2cc871e-3d07-4136-bea9-782d5a6e8dc9   
4  5c899683-d731-4c29-a7c0-715f5c8368da  d3a7f85b-0d5d-42b2-bae7-d221da5d861b   

  Transaction Date Product Category   Price  Quantity Payment Method  \
0       2024-04-13      Electronics  370.81       3.0         PayPal   
1       2023-07-10           Sports  119.51       4.0     Debit Card   
2       2025-01-29      Electronics  219.32       2.0         PayPal   
3       2023-03-25            Books   76.09       NaN     Debit Card   
4       2023-09-13   Home & Kitchen   59.01       1.0         PayPal   

  Shipping Status  Customer Rating  
0         Shipped         